# Classifying exoplanets with Kepler data

## Trevor Santiago

#### Can we predict whether a record is an exoplanet or not given certain measurements?

- How accurate can we do so?
- What measurements are the most important for our prediction?


[Data Source](https://www.kaggle.com/nasa/kepler-exoplanet-search-results)

In [1]:
!pip install -U scikit-learn
!pip install xgboost

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance
from sklearn import set_config

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, \
                            GradientBoostingClassifier, BaggingClassifier, \
                            StackingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/trevor-santiago/Kepler_project/main/cumulative.csv').drop(columns=['rowid'])

In [4]:
# non-useful and data leakage columns
non_inputs = [
    'kepid', 'kepoi_name', 'kepler_name', 
    'koi_score', 'koi_tce_delivname', 'koi_fpflag_nt', 
    'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
    'koi_disposition', 'koi_pdisposition'
]
non_inputs.extend([col for col in data.columns if '_err' in col])

# Set up features and label

Our target `koi_pdisposition` is a string with values `"CANDIDATE"` and `"FALSE POSITIVE"`, so we need to encode it.

In [5]:
X = data.copy()
y = data.koi_pdisposition.copy()
y = LabelEncoder().fit_transform(y)

### 3-way split data

These sets will stay the same throughout to accurately compare models. Model performance on the test set will be checked once at the end.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

# Modelling

For each of our models we will drop the data leakage/unimportant columns (see above), impute any missing values with the median of the respective columns, scale the data, and then fit the model. 

Since there are no categorical variables and we are interested in the importance of each feature at the end, we will keep this Pipeline mostly the same throughout.

Our main evaluation metric will be accuracy, but we will also take a look at log-loss to see the confidence of our models.

## Baseline Model


We'll start with a very basic model as a baseline to compare improvement of our future models. First we define our numeric and categorical columns for preprocessing steps. Then we define our main preprocessing pipelines that will be used throughout. Unimportant and data-leakage columns are discarded, then numeric and categorical columns are split to their respective sub-pipeline for processing.

In [7]:
num_cols = [
    'koi_srad', 'koi_period', 'koi_time0bk',
    'koi_impact', 'koi_duration', 'koi_depth',
    'koi_prad', 'koi_teq', 'koi_insol',
    'koi_model_snr', 'koi_steff', 'koi_slogg'
]

cat_cols = ['koi_kepmag', 'koi_tce_plnt_num']

# Need to convert the above to indexes for ColumnTransformer
num_cols = list(map(lambda x: int(np.where(data.columns == x)[0][0]), num_cols))
cat_cols = list(map(lambda x: int(np.where(data.columns == x)[0][0]), cat_cols))

In [8]:
num_prep = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('scale', RobustScaler())
])

# Bin koi_kepmag
binner = ColumnTransformer([('bin', KBinsDiscretizer(), [0])], remainder='passthrough')
cat_prep = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('bin', binner)
])

prepper = ColumnTransformer([
    ('numeric', num_prep, num_cols),
    ('categorical', cat_prep, cat_cols)
], remainder='drop')

In [9]:
model = Pipeline([
    ('preprocess', prepper),
    ('clf', KNeighborsClassifier())
])

In [10]:
model.fit(X_train, y_train)

# Use prediction probabilities for log-loss
y_pred_probs = model.predict_proba(X_val)

# Get predictions from probabilities above
y_pred = np.argmax(y_pred_probs, axis=1)

loss = log_loss(y_val, y_pred_probs)
acc = accuracy_score(y_val, y_pred)
print(f'log loss: {loss:.5f}')
print(f'Accuracy: {acc*100:.1f}%')

log loss: 1.75887
Accuracy: 79.4%


This basic model gives decent accuracy, but the log-loss is quite high which can mean it is either too confident in its incorrect predictions or not very confident in its correct predictions. Now we will try to improve both of these metrics

## Model improvement

First we will try bagging on the `KNeighborsClassifier` from the baseline

In [11]:
knn = KNeighborsClassifier()
bag = BaggingClassifier(knn,
                        n_estimators=20,
                        oob_score=True,
                        n_jobs=-1)
model = Pipeline([
    ('preprocess', prepper),
    ('clf', bag)
])

In [12]:
model.fit(X_train, y_train)

y_pred_probs = model.predict_proba(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)

loss = log_loss(y_val, y_pred_probs)
acc = accuracy_score(y_val, y_pred)
print(f'log loss: {loss:.5f}')
print(f'Accuracy: {acc*100:.1f}%')

/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:640: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:644: RuntimeWarning: invalid value encountered in true_divide
  oob_decision_function = (predictions /


log loss: 1.05445
Accuracy: 80.7%


We get about the same level of accuracy, but a good improvement in log-loss. Now let's try a couple other simple yet common models

Logistic Regression:

In [13]:
model = Pipeline([
    ('preprocess', prepper),
    ('clf', LogisticRegression(max_iter=500))
])

model.fit(X_train, y_train)
y_pred_probs = model.predict_proba(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)

loss = log_loss(y_val, y_pred_probs)
acc = accuracy_score(y_val, y_pred)
print(f'log loss: {loss:.5f}')
print(f'Accuracy: {acc*100:.1f}%')

log loss: 0.54592
Accuracy: 74.8%


Worse accuracy but much better log-loss.

Decision Tree:

In [14]:
model = Pipeline([
    ('preprocess', prepper),
    ('clf', DecisionTreeClassifier())
])

model.fit(X_train, y_train)
y_pred_probs = model.predict_proba(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)

loss = log_loss(y_val, y_pred_probs)
acc = accuracy_score(y_val, y_pred)
print(f'log loss: {loss:.5f}')
print(f'Accuracy: {acc*100:.1f}%')

log loss: 7.48918
Accuracy: 78.3%


Slightly better accuracy than Logistic but horrendous log-loss. Let's try voting with these 3 models

In [15]:
voter = VotingClassifier(estimators=[
    ('knn', KNeighborsClassifier()),
    ('logistic', LogisticRegression(max_iter=500)),
    ('dtree', DecisionTreeClassifier())
], voting='soft')

In [16]:
model = Pipeline([
    ('preprocess', prepper),
    ('clf', voter)
])

model.fit(X_train, y_train)
y_pred_probs = model.predict_proba(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)

loss = log_loss(y_val, y_pred_probs)
acc = accuracy_score(y_val, y_pred)
print(f'log loss: {loss:.5f}')
print(f'Accuracy: {acc*100:.1f}%')

log loss: 0.45825
Accuracy: 80.5%


Not bad. Accuracy of KNN with an improvement on Logistic's log-loss. Now let's try some stronger algorithms.

### More Robust models

We will try 3 ensembling models. To save some work, we'll wrap these into a cross-validation with hyperparameter search.

In [17]:
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass

In [18]:
model = Pipeline([
    ('preprocess', prepper),
    ('clf', DummyEstimator())
])

search_space = [
    {
        'clf': [ExtraTreesClassifier()],
        'clf__n_estimators': range(50, 301, 50),
        'clf__min_samples_leaf': [1, 2],
        'clf__criterion': ['gini', 'entropy']
    },
    {
        'clf': [RandomForestClassifier()],
        'clf__n_estimators': range(50, 301, 50),
        'clf__min_samples_leaf': [1, 2],
        'clf__max_features': ['sqrt', 'log2', 0.5],
        'clf__criterion': ['gini', 'entropy']
    },
    {
        'clf': [GradientBoostingClassifier()],
        'clf__n_estimators': range(50, 301, 50),
        'clf__max_depth': list(range(3, 6))+[None],
        'clf__subsample': [0.8, 0.9, 1.0],
        'clf__loss': ['deviance', 'exponential']        
    }
]


clf_algos_rand = RandomizedSearchCV(estimator=model, 
                                    param_distributions=search_space, 
                                    n_iter=25,
                                    cv=5, 
                                    n_jobs=-1,
                                    verbose=1,
                                    scoring='accuracy')

clf_algos_rand.fit(X_train, y_train)
print('Best estimator: ', clf_algos_rand.best_estimator_['clf'])

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best estimator:  RandomForestClassifier(criterion='entropy', max_features='log2',
                       min_samples_leaf=2)


Now we use the best estimator from above and see how it performs.

In [19]:
y_pred_probs = clf_algos_rand.best_estimator_.predict_proba(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)

loss = log_loss(y_val, y_pred_probs)
acc = accuracy_score(y_val, y_pred)
print(f'log loss: {loss:.5f}')
print(f'Accuracy: {acc*100:.1f}%')

log loss: 0.40096
Accuracy: 83.8%


Pretty good! Nice jump in accuracy, and still lower log-loss. How about using this as a meta-learner in a stack? In order to make this work we need to define individual pipelines for each of the base estimators. Then we pass those in along with this meta-learner to a `StackingClassifier`. Let's see how it performs.

In [20]:
knn_pipe = Pipeline([
    ('preprocess', prepper),
    ('kneighbors', KNeighborsClassifier())
])

lr_pipe = Pipeline([
    ('preprocess', prepper),
    ('reg', LogisticRegression(max_iter=500))
])

dtree_pipe = Pipeline([
    ('preprocess', prepper),
    ('tree', DecisionTreeClassifier())
])

# Collect all base learner pipes to send into stack
base_estimators = [
    ('knn', knn_pipe),
    ('lr', lr_pipe),
    ('dtree', dtree_pipe)
]

# Create stack, passing original data on to meta-learner
stack = StackingClassifier(estimators=base_estimators,
                           final_estimator=clf_algos_rand.best_estimator_,
                           passthrough=True,
                           n_jobs=-1)

In [21]:
stack.fit(X_train, y_train)
y_pred_probs = stack.predict_proba(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)

loss = log_loss(y_val, y_pred_probs)
acc = accuracy_score(y_val, y_pred)
print(f'log loss: {loss:.5f}')
print(f'Accuracy: {acc*100:.1f}%')

log loss: 0.32989
Accuracy: 85.8%


Nice! Fair improvement to both accuracy and log-loss. Finally, we'll give `XGBoost` a shot.

In [22]:
model = Pipeline([
    ('preprocess', prepper),
    ('clf', XGBClassifier(eval_metric='error', use_label_encoder=False, n_jobs=-1))
])

model.fit(X_train, y_train)
y_pred_probs = model.predict_proba(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)

loss = log_loss(y_val, y_pred_probs)
acc = accuracy_score(y_val, y_pred)
print(f'log loss: {loss:.5f}')
print(f'Accuracy: {acc*100:.1f}%')

log loss: 0.42927
Accuracy: 82.6%


Not terrible, but not worth it over the stack. We'll determine the stack as our final model.

## Final Model

In [23]:
# Redefine stack for refitting on all training & validation data
final_stack = StackingClassifier(estimators=base_estimators,
                                 final_estimator=clf_algos_rand.best_estimator_,
                                 passthrough=True,
                                 n_jobs=-1)

In [24]:
X = pd.concat([X_train, X_val])
y = LabelEncoder().fit_transform(y=X.koi_pdisposition.copy())

### Fit on all data and eval on test set

In [25]:
final_stack.fit(X, y)
y_pred_probs = final_stack.predict_proba(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

loss = log_loss(y_test, y_pred_probs)
acc = accuracy_score(y_test, y_pred)
print(f'log loss: {loss:.5f}')
print(f'Accuracy: {acc*100:.1f}%')

log loss: 0.33312
Accuracy: 85.6%


Awesome! Now let's see what features helped us get here.

### Check which features were most important

In [26]:
r = permutation_importance(final_stack, X, y, n_repeats=30, n_jobs=-1)

In [27]:
num_cols = [
    'koi_srad', 'koi_period', 'koi_time0bk',
    'koi_impact', 'koi_duration', 'koi_depth',
    'koi_prad', 'koi_teq', 'koi_insol',
    'koi_model_snr', 'koi_steff', 'koi_slogg'
]

cat_cols = ['koi_kepmag', 'koi_tce_plnt_num']
inputs = num_cols + cat_cols
pd.DataFrame(data=r.importances_mean, index=X.columns).loc[inputs,:].sort_values(0, ascending=False)

,0
koi_prad,0.116966
koi_duration,0.093745
koi_model_snr,0.077899
koi_period,0.067117
koi_time0bk,0.035220
koi_depth,0.030694
koi_slogg,0.029760
koi_impact,0.027966
koi_steff,0.023677
koi_insol,0.023403


### Final model architecture:

In [28]:
set_config(display='diagram')

final_stack

StackingClassifier(estimators=[('knn',
                                Pipeline(steps=[('preprocess',
                                                 ColumnTransformer(transformers=[('numeric',
                                                                                  Pipeline(steps=[('imp',
                                                                                                   SimpleImputer(strategy='median')),
                                                                                                  ('scale',
                                                                                                   RobustScaler())]),
                                                                                  [43,
                                                                                   10,
                                                                                   13,
                                                                                   16,
                                                                                   19,
                                                                                   22,
                                                                                   25,
                                                                                   28,
                                                                                   31,
                                                                                   34,
                                                                                   37,
                                                                                   40]),
                                                                                 ('categorical',
                                                                                  Pipeline(steps=[('imp',
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('bin',
                                                                                                   ColumnTransformer(remainder='p...
                                                                                                      RobustScaler())]),
                                                                                     [43,
                                                                                      10,
                                                                                      13,
                                                                                      16,
                                                                                      19,
                                                                                      22,
                                                                                      25,
                                                                                      28,
                                                                                      31,
                                                                                      34,
                                                                                      37,
                                                                                      40]),
                                                                                    ('categorical',
                                                                                     Pipeline(steps=[('imp',
                                                                                                      SimpleImputer(strategy='most_frequent')),
                                                                                                     ('bin',
                                             